<a href="https://colab.research.google.com/github/RonildoSilva/datasets/blob/main/big_black_forecast_da_rnn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from datetime import datetime

! pip install flood-forecast

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 117 kB 4.1 MB/s 
     |████████████████████████████████| 201 kB 23.4 MB/s 
     |████████████████████████████████| 28.8 MB 1.4 MB/s 
     |████████████████████████████████| 564 kB 66.3 MB/s 
     |████████████████████████████████| 2.8 MB 27.1 MB/s 
     |████████████████████████████████| 1.1 MB 71.3 MB/s 
     |████████████████████████████████| 1.8 MB 59.5 MB/s 
     |████████████████████████████████| 5.9 MB 38.7 MB/s 
     |████████████████████████████████| 158 kB 48.2 MB/s 
     |████████████████████████████████| 181 kB 52.8 MB/s 
     |████████████████████████████████| 63 kB 1.4 MB/s 
     |████████████████████████████████| 3.5 MB 32.4 MB/s 
     |████████████████████████████████| 157 kB 62.9 MB/s 
     |████████████████████████████████| 157 kB 45.2 MB/s 
     |████████████████████████████████| 157 kB 47.7 MB/s 
     |████████████████████████████

In [2]:
! pip install wget

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9675 sha256=492703ccc06272cc27fb69bb7a31c4e93b9250c6ab85df6812ed2da742135cf4
  Stored in directory: /root/.cache/pip/wheels/a1/b6/7c/0e63e34eb06634181c63adacca38b79ff8f35c37e3c13e3c02
Successfully built wget


In [4]:
import wget
import os.path

In [ ]:
#! unzip 01010500FVE_flow.csv.zip

In [7]:
import wget
import os.path

if(not os.path.exists('df_train_keras_tk.csv')):
    wget.download('https://raw.githubusercontent.com/RonildoSilva/datasets/main/df_train_keras_tk.csv')

if(not os.path.exists('df_val_keras_tk.csv')):
    wget.download('https://raw.githubusercontent.com/RonildoSilva/datasets/main/df_val_keras_tk.csv')

if(not os.path.exists('df_test_keras_tk.csv')):
    wget.download('https://raw.githubusercontent.com/RonildoSilva/datasets/main/df_test_keras_tk.csv')

Merge

In [8]:
import pandas as pd 
df = pd.concat([
    pd.read_csv('df_train_keras_tk.csv'),
    pd.read_csv('df_val_keras_tk.csv'),
    pd.read_csv('df_test_keras_tk.csv')
])

In [9]:
print(
    pd.read_csv('df_train_keras_tk.csv').shape,
    pd.read_csv('df_val_keras_tk.csv').shape,
    pd.read_csv('df_test_keras_tk.csv').shape
)

(12363, 20) (4021, 20) (4020, 20)


In [13]:
TRAIN_START = 0
TRAIN_END = pd.read_csv('df_train_keras_tk.csv').shape[0]

VAL_START = TRAIN_END + 1
VAL_END = VAL_START + pd.read_csv('df_val_keras_tk.csv').shape[0]

TEST_START = VAL_END + 1
TEST_END = TEST_START + pd.read_csv('df_test_keras_tk.csv').shape[0] - 1

In [14]:
print(TRAIN_START, TRAIN_END, VAL_START, VAL_END, TEST_START, TEST_END)

0 12363 12364 16385 16386 20405


In [15]:
df['datetime'] = pd.to_datetime(df['datetime']).dt.strftime('%Y-%m-%d')
#df['datetime'] = pd.to_datetime(df['datetime']).dt.strftime('%Y-%m-%d %H:%M:%S+00:00')
#2000-04-04 04:00:00+00:00

In [16]:
df = df[["TimeConclusion", "Duration", "datetime"]]
df.to_csv('df_full.csv', index=False)

In [17]:
df.head()

,TimeConclusion,Duration,datetime
0,31.0087,0.0000,2012-10-09
1,15.0003,16.0084,2012-10-09
2,15.0002,0.0001,2012-10-12
3,0.0000,15.0002,2012-10-25
4,30.9822,0.0000,2012-11-09


In [ ]:
"""
!pip install -Uqq ipdb
import ipdb
%pdb on
"""

In [18]:
df.columns

Index(['TimeConclusion', 'Duration', 'datetime'], dtype='object')

Config

In [19]:
def make_config_file(flow_file_path, gage_id, station_id, weight_path=None, pretrained=[]):
    run = wandb.init(project="github_aistream-peelout_flow-forecast")
    wandb_config = run.config
    the_wandb_c = run.config
    print(wandb_config)
    the_config4 = {"model_name": "DARNN",
        "model_type": "PyTorch",
        "model_params": {
        "n_time_series": 4,
        "forecast_history": 14,
        "hidden_size_encoder":32,
        "decoder_hidden_size": 32,
        "out_feats": 1,
        "dropout": 0.1,
        "gru_lstm": False},
       "dataset_params":{"class": "default",
       "num_workers":2,
       "pin_memory": True,
       "training_path": flow_file_path,
       "validation_path": flow_file_path,
       "test_path": flow_file_path,
       "batch_size":200,
       "forecast_history": (14 - 1),
       "forecast_length": 1,
       "scaler": "StandardScaler",
       "train_start": TRAIN_START,
       "train_end": TRAIN_END,
       "valid_start": VAL_START,
       "valid_end": VAL_END,
       "sort_column": "datetime",# "hour_updated",
       "test_start": TEST_START,
       "test_end": TEST_END,
       "target_col": ["TimeConclusion"],
       "relevant_cols": ["TimeConclusion", "Duration"], 
       "interpolate":{
           "method":"back_forward_generic",
           "params":{
               "relevant_columns":["TimeConclusion", "Duration"]}
           },
      "feature_param":
          {
            "datetime_params":{
            "day":"numerical",
            "month": "numerical"
            }
          }
       },
    "training_params":
    {
       "criterion":"MSE",
       "optimizer": "SGD",
       #"criterion_params":{"baseline_method":"mean"},
    "optim_params":{
       "lr": 0.001
    },
       "epochs": 12,
       "batch_size": 200
    },
    "early_stopping":{
        "patience":3
    },
    "GCS": False,
    "sweep":True,
    "wandb":False,
    "forward_params":{},
   "metrics":["MSE", "MAPE"],
   "inference_params":
   {     
          "datetime_start":"2012-03-02",#"2020-05-31",
          "hours_to_forecast":336, 
          "num_prediction_samples": 20,
          "test_csv_path":flow_file_path,
          "decoder_params":{
            "decoder_function": "simple_decode", 
            "unsqueeze_dim": 1},
          "dataset_params":{
             "file_path": flow_file_path,
             "sort_column": "datetime", #"hour_updated",
             "scaling": "StandardScaler",
             "forecast_history": (14 - 1),
             "forecast_length":1,
             "relevant_cols": ["TimeConclusion", "Duration"],
             "target_col": ["TimeConclusion"],
             "interpolate_param":{
                 "method":"back_forward_generic",
                 "params":{"relevant_columns":["TimeConclusion", "Duration"]}
                 }, 
            "feature_params":
        {
         "datetime_params":{
            "day":"numerical",
            "month": "numerical"
         
     }
             }
          }
          } 
    }

      
    if weight_path:
        the_config4["weight_path"] = weight_path
    wandb.config.update(the_config4)
    print("config made")
    return the_config4
  
wandb_sweep_config_full = {
  "name": "Default sweep",
  "method": "grid",
  "parameters": {
        "forecast_length":{
            "values":[1]}
            ,
        "batch_size": {
            "values": [200]#, 400, 600]
        },
        "lr":{
            "values":[0.001]#, 0.0001, .01]
        },
        "forecast_history":{
            "values":[14]#, 20, 24, 48]
        },
        "out_seq_length":{
            "values":[1]
        },
        "hidden_encoder":
        {
            "values":[32]#, 64, 128, 256]
        },
        "hidden_decoder":
        {
            # 32, 64, 128 
            "values":[32]#, 64, 128, 256]
        }, 
        "dropout":
        {
            "values": [0.1],#, 0.3, 0.5, 0.8]
        }
  
    }
}

In [20]:
import wandb
from flood_forecast.trainer import train_function


sweep_id = wandb.sweep(wandb_sweep_config_full, project="github_aistream-peelout_flow-forecast")
#sweep_id = "03l75ilz"

#file_path = "01010500FVE_flow.csv"
file_path = "df_full.csv"

#wandb.agent(sweep_id, lambda: train_function("PyTorch", make_config_file(file_path, "01010500", "FVE", None)))

model = train_function("PyTorch", make_config_file(file_path, "01010500", "FVE", None))

ERROR:wandb.jupyter:Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


<IPython.core.display.Javascript object>

wandb: You can find your API key in your browser here: https://wandb.ai/authorize


wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit: ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Create sweep with ID: o2lcrwmp
Sweep URL: https://wandb.ai/ronildosilva/github_aistream-peelout_flow-forecast/sweeps/o2lcrwmp


wandb: Currently logged in as: ronildosilva. Use `wandb login --relogin` to force relogin


{}
config made
interpolate should be below
Running code to add temporal features
Created datetime feature columns are: 
['day', 'month']
scaling now
interpolate should be below
Running code to add temporal features
Created datetime feature columns are: 
['day', 'month']
scaling now
interpolate should be below
Running code to add temporal features
Created datetime feature columns are: 
['day', 'month']
scaling now
Using Wandb config:
{'model_name': 'DARNN', 'model_type': 'PyTorch', 'model_params': {'n_time_series': 4, 'forecast_history': 14, 'hidden_size_encoder': 32, 'decoder_hidden_size': 32, 'out_feats': 1, 'dropout': 0.1, 'gru_lstm': False}, 'dataset_params': {'class': 'default', 'num_workers': 2, 'pin_memory': True, 'training_path': 'df_full.csv', 'validation_path': 'df_full.csv', 'test_path': 'df_full.csv', 'batch_size': 200, 'forecast_history': 13, 'forecast_length': 1, 'scaler': 'StandardScaler', 'train_start': 0, 'train_end': 12363, 'valid_start': 12364, 'valid_end': 16385, 'so

Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.
Using a non-full backward hook when outputs are generated by different autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_output. Please use register_full_backward_hook to get the documented behavior.
Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  ../c10/core/TensorImpl.h:1405.)
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:1053: UserWarning:

Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input

plotting with CI now
Now plotting final plots
